<a href="https://colab.research.google.com/github/KeniaEsparza/TI_1_Practica3_EsparzaKenia/blob/main/TI_1_Practica3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 3

**Nombre:** Esparza Duran Kenia Jaqueline

**e-mail:** kenia.esparza5866@alumnos.udg.mx

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## MODULES

In [71]:
import math
import numpy as np
import pandas as pd

import plotly.graph_objects as go

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

from scipy.spatial import distance

## CLASSES

In [14]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

## FUNCTIONS

In [15]:
###############################################################################################
# Turning angle
# This function calculates the turning angle between three consecutive positions
###############################################################################################
def turning_angle(pos_a, pos_b, pos_c):
    """
    Arguments:
        pos_a: First position coordinates
        pos_b: Second position coordinates
        pos_c: Third position coordinates
    Returns:
        theta: Turning angle
    """
    vec_ab = Vec2d(pos_b.x - pos_a.x, pos_b.y - pos_a.y)
    norm_ab = vec_ab.get_length()

    vec_bc = Vec2d(pos_c.x - pos_b.x, pos_c.y - pos_b.y)
    norm_bc = vec_bc.get_length()

    dot_p = vec_ab.x * vec_bc.x + vec_ab.y * vec_bc.y

    # Nota: Evitar division por cero con np.finfo(float).eps
    cos_theta = dot_p / (norm_ab * norm_bc + np.finfo(float).eps)

    # Angle orientation
    cross_p = vec_ab.x * vec_bc.y - vec_ab.y * vec_bc.x
    orient = np.sign(cross_p)
    if orient == 0:
        orient = 1

    theta = np.arccos(np.around(cos_theta,4)) * orient
    return theta



## Actividad 1: Path Length - (BM1 vs BM2 vs CRW) (4 pts)

* Implementar función que genere **Brownian Motions** (BM) utilizando **pandas**.
* Implementar función que genere **Correlated Random Walks** (CRW) utilizando pandas.
* Implementar una función alternativa a las ya disponibles en los distintos modulos de python que calcule los valores de la curva de **path length** de una trayectoria.
* Guardar los valores de la métrica en un Data Frame de **pandas**.
* Visualizar con **plotly**.

In [22]:
# Load existing trajectories to test your implementation
# BM speed = 3
BM_2d_df_3 = pd.read_csv('/trajectories/brownian_3.csv') #/trajectories/brownian_3.csv

# Load existing trajectories to test your implementation
# BM speed = 6
BM_2d_df_6 = pd.read_csv('/trajectories/crw_6_6.csv')

# Load existing trajectories to test your implementation
CRW_2d_df_9 = pd.read_csv('/trajectories/crw_6_9.csv')

In [88]:
# Define your function to compute path length for given trajectory
def path_length_curve(df):
  distances = [distance.euclidean(df.iloc[i-1][:2], df.iloc[i][:2]) for i in range(1,len(df))]
  return np.cumsum(distances)

pl_bm3 = path_length_curve(BM_2d_df_3)
pl_bm6 = path_length_curve(BM_2d_df_6)
pl_crw9 = path_length_curve(CRW_2d_df_9)

fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(pl_bm3)), y=pl_bm3, mode='lines', name='BM3 Path Length'))
fig.add_trace(go.Scatter(x=np.arange(len(pl_bm6)), y=pl_bm6, mode='lines', name='BM6 Path Length'))
fig.add_trace(go.Scatter(x=np.arange(len(pl_crw9)), y=pl_crw9, mode='lines', name='CRW9 Path Length'))
fig.update_layout(title='Path Length Curves', xaxis_title='Step', yaxis_title='Path Length')
fig.show()

In [90]:
def calculate_path_length(trajectory):
    distances = [distance.euclidean(trajectory.iloc[i-1,:2], trajectory.iloc[i,:2]) for i in range(1,len(trajectory))]
    path_length = np.cumsum(distances)
    path_length_df = pd.DataFrame({'Step': np.arange(1,len(path_length)+1), 'Path Length': path_length})
    return path_length_df

path_length_df_bm3 = calculate_path_length(BM_2d_df_3)

# Get Path length calling the function
PL_BM_3 = calculate_path_length(BM_2d_df_3)

# Get Path length calling the function
PL_BM_6 = calculate_path_length(BM_2d_df_6)

# Get Path length calling the function
PL_CRW_6 = calculate_path_length(CRW_2d_df_9)

In [92]:
# Plotting
# Init figure
fig_path_length = go.Figure()

# First trace BM1
fig_path_length.add_trace(go.Scatter(
    x=np.arange(len(PL_BM_3)) + 1,
    y=PL_BM_3,
    name='Path Length BM3',
    line=dict(color='blue', width=3),  # Estilo de línea azul
    showlegend=True
))

# Second trace BM2
fig_path_length.add_trace(go.Scatter(
    x=np.arange(len(PL_BM_6)) + 1,
    y=PL_BM_6,
    name='Path Length BM6',
    line=dict(color='orange', width=4),
    showlegend=True
))

# Third trace CRW
fig_path_length.add_trace(go.Scatter(
    x=np.arange(len(PL_CRW_6)) + 1,
    y=PL_CRW_6,
    name='Path Length CRW6',
    line=dict(color='purple', width=3),  # Estilo de línea morada
    showlegend=True
))
fig_path_length.update_layout(
    title='Comparación de Longitud de Trayectoria',
    xaxis_title='Paso',
    yaxis_title='Longitud de Trayectoria',
    legend_title='Tipo de Trayectoria',

    plot_bgcolor="white",  # Fondo blanco para mejorar contraste
    xaxis=dict(showline=True, showgrid=False, linecolor='black'),  # Estilo del eje X
    yaxis=dict(showline=True, showgrid=True, gridcolor='lightgrey', linecolor='black')  # Estilo del eje Y
)
fig_path_length.show()

## Actividad 2: Mean Squared Displacement - (Brownian vs CRW) (4 pts)

* Generar una trayectoria tipo **BM** y una **CRW**.
* Implementar una función que calcule los valores de la curva de **mean squared displacement** de una trayectoria.
* Guardar metricas en Pandas Data Frame.
* Visualizar con **plotly**.

In [ ]:
# Load existing trajectories to test your implementation
# BM speed = 6
BM_2d_df_6 = pd.read_csv('trajectories/brownian_6.csv')

# Load existing trajectories to test your implementation
# CRW speed = 6, c = 0.9
CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')

In [ ]:
# Show trajectories
# Init figure
fig_3d = go.Figure()

# Plot trajectory in 3-D space
fig_3d.add_trace(
    go.Scatter3d(x = BM_2d_df_6.x_pos,
                 y = BM_2d_df_6.y_pos,
                 z = BM_2d_df_6.index,
                 marker = dict(size=2),
                 line = dict(color='blue', width=2),
                 mode = 'lines',
                 name = 'BM 2d',
                 showlegend = True))


fig_3d.add_trace(
    go.Scatter3d(x = CRW_2d_df_9.x_pos,
                 y = CRW_2d_df_9.y_pos,
                 z = CRW_2d_df_9.index,
                 marker = dict(size=2),
                 line = dict(color='red', width=2),
                 mode = 'lines',
                 name = 'CRW 2d',
                 showlegend = True))

fig_3d.show()

In [ ]:
# Define your function to compute Mean Squared Displacement for given trajectory

## start - Add your code here
# def function (trajectory):


    #return MSD_df

## end - Add your code here

In [ ]:
# Get Mean Squared Displacement calling the function
MSD_BM = # Add your code here function(BM_2d_df_6)

# Get Mean Squared Displacement calling the function
MSD_CRW = # Add your code here function(CRW_2d_df_9)

In [ ]:
# Init figure
fig_path_length = go.Figure()

# first trace MSD_BM
## start - Add your code here

## end - Add your code here


# Second trace MSD_CRW
## start - Add your code here

## end - Add your code here


fig_path_length.show()

## Actividad 3: Turning-angle Distribution - (Dist. origen vs Dist. observada) (6 pts)

* Generar **CRWs** con dos exponentes diferentes
* Guardar trayectorias en **Pandas** Data Frame
* Implementar una función que calcule los valores de **turning angle** de una trayectoria.
* Comparar en gráfica distribución origen vs distribución observada (Histograma)
* Visualizar con **plotly**

In [34]:
# Load existing trajectories to test your implementation
# CRW speed = 6,
# wrapcauchy [c = 0.6]
CRW_2d_df_6 = pd.read_csv('/trajectories/crw_6_6.csv') #/trajectories

# Load existing trajectories to test your implementation
# CRW speed = 6,
# wrapcauchy [c = 0.9]
CRW_2d_df_9 = pd.read_csv('/trajectories/crw_6_9.csv')

In [35]:
# aux to store turning angles
aux_ta_CRW_2d_df_6 = np.empty(shape=(0))


# Iterate over trajectory compute turning angles
for index, row in CRW_2d_df_6[1:-1].iterrows():
    pos_a = Vec2d(CRW_2d_df_6.iloc[index-1])
    pos_b = Vec2d(CRW_2d_df_6.iloc[index])
    pos_c = Vec2d(CRW_2d_df_6.iloc[index+1])
    angle = turning_angle(pos_a,pos_b,pos_c)
    aux_ta_CRW_2d_df_6 = np.append(aux_ta_CRW_2d_df_6,angle)

# aux to store turning angles
aux_ta_CRW_2d_df_9 = np.empty(shape=(0))


for index in range(1, len(CRW_2d_df_9)-1):
    pos_a = Vec2d(CRW_2d_df_9.iloc[index-1])
    pos_b = Vec2d(CRW_2d_df_9.iloc[index])
    pos_c = Vec2d(CRW_2d_df_9.iloc[index+1])
    angle = turning_angle(pos_a, pos_b, pos_c)
    aux_ta_CRW_2d_df_9 = np.append(aux_ta_CRW_2d_df_9, angle)

In [36]:
# Define your function to compute Turning Angles for given trajectory
def compute_turning_angles(trajectory):
    ta_values = []

    for i in range(1,len(trajectory)-1):
        pos_a = Vec2d(trajectory.iloc[index-1])
        pos_b = Vec2d(trajectory.iloc[index])
        pos_c = Vec2d(trajectory.iloc[index+1])
        angle = turning_angle(pos_a, pos_b, pos_c)
        ta_values.append(angle)
    return pd.DataFrame(ta_values,columns=['Turning Angle'])

In [56]:
# Get Turning Angles calling the function
ta_CRW_2d_df_6 = compute_turning_angles(CRW_2d_df_6)

# Get Turning Angles calling the function
ta_CRW_2d_df_9 = compute_turning_angles(CRW_2d_df_9)

# Función para generar la distribución de origen
def compute_origin_distribution(c, num_samples):
    return wrapcauchy.rvs(c, size=num_samples)


In [62]:
# Check documentation
#https://plotly.com/python/histograms/

# PLot histogram
fig_met_df_3 = go.Figure()


# Histogram turning angle CRW_2d_df_6
fig_met_df_3.add_trace(go.Histogram(x=ta_CRW_2d_df_6['Turning Angle'], name='CRW_2d_df_6'))


# Histogram turning angle CRW_2d_df_9
fig_met_df_3.add_trace(go.Histogram(x=ta_CRW_2d_df_9['Turning Angle'], name='CRW_2d_df_9'))


# Add origin distributions
# Generar distribuciones de origen para CRW_2d_df_6 y CRW_2d_df_9
num_samples = 1000
origin_distribution_6 = wrapcauchy.rvs(0.6, size=num_samples)
origin_distribution_9 = wrapcauchy.rvs(0.9, size=num_samples)

# Histograma de la distribución de origen para CRW_2d_df_6
fig_met_df_3.add_trace(go.Histogram(x=origin_distribution_6, name='Origin Distribution CRW_2d_df_6'))

# Histograma de la distribución de origen para CRW_2d_df_9
fig_met_df_3.add_trace(go.Histogram(x=origin_distribution_9, name='Origin Distribution CRW_2d_df_9'))




## Actividad 4: Step-length Distribution - (Dist. origen vs Dist. observada) (6 pts)

* Implementar función que genere **Lévy Walks** (LW) utilizando pandas.
* Guardar trayectorias en Pandas Data Frame.
* Implementar una función que calcule los valores de **step lenght** de una trayectoria.
* Guardar trayectorias en **pandas** Data Frame.
* Obtener **Step-length** distribution.
* Comparar en gráfica distribución origen vs distribución observada (Histograma).
* Visualizar con plotly.

In [68]:
# Load existing trajectories to test your implementation
# Levy speed = 6
# levy_stable [alpha=1.0, beta=1.0, loc=3.0]
Levy_2d_df_1 = pd.read_csv('/trajectories/levy_6_1.csv')

# Load existing trajectories to test your implementation
# Levy speed = 6
# levy_stable [alpha=0.7, beta=1.0, loc=3.0]
Levy_2d_df_7 = pd.read_csv('/trajectories/levy_6_7.csv')

In [74]:
# Define your function to compute Step lengths for given trajectory

## start - Add your code here
def compute_step_lengths(trajectory):
    steps = []
    for i in range(len(trajectory) - 1):
        pos_a = Vec2d(trajectory.iloc[i]['x_pos'], trajectory.iloc[i]['y_pos'])
        pos_b = Vec2d(trajectory.iloc[i + 1]['x_pos'], trajectory.iloc[i + 1]['y_pos'])
        step_length = distance.euclidean((pos_a.x, pos_a.y), (pos_b.x, pos_b.y))
        steps.append(step_length)
    return pd.DataFrame(steps, columns=['Step Length'])

## end - Add your code here

In [75]:
# Get Step lengths calling the function
sl_Levy_2d_df_1 = compute_step_lengths(Levy_2d_df_1)

# Get Step lengths calling the function
sl_Levy_2d_df_7 = compute_step_lengths(Levy_2d_df_7)


In [76]:
# PLot histogram
fig_met_df_4 = go.Figure()

# Histogram step-length Levy_2d_df_1
## start - Add your code here
fig_met_df_4.add_trace(go.Histogram(x=sl_Levy_2d_df_1['Step Length'], name='Levy_2d_df_1'))
## end - Add your code here


# Histogram step-length Levy_2d_df_7
## start - Add your code here
fig_met_df_4.add_trace(go.Histogram(x=sl_Levy_2d_df_7['Step Length'], name='Levy_2d_df_7'))
## end - Add your code here


# Add origin distributions
## start - Add your code here

## end - Add your code here


origin_distribution = np.random.normal(loc=0, scale=1, size=1000)  # Mean = 0, Standard Deviation = 1

# Add origin distribution to the plot
fig_met_df_4.add_trace(go.Histogram(x=origin_distribution, name='Origin Distribution'))

# Add legend
fig_met_df_4.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

# Update layout
fig_met_df_4.update_layout(
    title="Step-length Distribution",
    xaxis_title="Step Length",
    yaxis_title="Frequency"
)

# Show plot
fig_met_df_4.show()